In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from datetime import datetime
from plotnine import*
from matplotlib.font_manager import findfont
from pymongo import MongoClient
import time

In [2]:
def mongodb(database, table, username, password):    ### 匯入資料庫
#     client = MongoClient('localhost', 27017)
    client = MongoClient('163.14.137.58',8099, username=f'{username}', password=f'{password}')     ## 老師server
    time.sleep(3)
    db = client[f'{database}']                ## simi:database名稱
    collection_currency = db[f'{table}']      ## ptt:table名稱
    time.sleep(3)
    data = pd.DataFrame(data = collection_currency.find())    
    
    return data
# ptt = mongodb('text_mining', 'ptt_wine','simi','simi')

In [3]:
class TfidfEmbeddingVectorizer(object):
	def __init__(self, word_model):

		self.word_model = word_model
		self.word_idf_weight = None
		self.vector_size = word_model.wv.vector_size

	def fit(self, docs):  # comply with scikit-learn transformer requirement
		"""
		Fit in a list of docs, which had been preprocessed and tokenized,
		such as word bi-grammed, stop-words removed, lemmatized, part of speech filtered.
		Then build up a tfidf model to compute each word's idf as its weight.
		Noted that tf weight is already involved when constructing average word vectors, and thus omitted.
		:param
			pre_processed_docs: list of docs, which are tokenized
		:return:
			self
		"""

		text_docs = []
		for doc in docs:
			text_docs.append(" ".join(doc))

		tfidf = TfidfVectorizer()
		tfidf.fit(text_docs)  # must be list of text string

		# if a word was never seen - it must be at least as infrequent
		# as any of the known words - so the default idf is the max of
		# known idf's
		max_idf = max(tfidf.idf_)  # used as default value for defaultdict
		self.word_idf_weight = defaultdict(lambda: max_idf,
						   [(word, tfidf.idf_[i]) for word, i in tfidf.vocabulary_.items()])
		return self


	def transform(self, docs):  # comply with scikit-learn transformer requirement
		doc_word_vector = self.word_average_list(docs)
		return doc_word_vector


	def word_average(self, sent):
		"""
		Compute average word vector for a single doc/sentence.
		:param sent: list of sentence tokens
		:return:
			mean: float of averaging word vectors
		"""

		mean = []
		for word in sent:
			if word in self.word_model.wv.vocab:
				mean.append(self.word_model.wv.get_vector(word) * self.word_idf_weight[word])  # idf weighted

		if not mean:  # empty words
			# If a text is empty, return a vector of zeros.
			logging.warning("cannot compute average owing to no vector for {}".format(sent))
			return np.zeros(self.vector_size)
		else:
			mean = np.array(mean).mean(axis=0)
			return mean


	def word_average_list(self, docs):
		"""
		Compute average word vector for multiple docs, where docs had been tokenized.
		:param docs: list of sentence in list of separated tokens
		:return:
			array of average word vector in shape (len(docs),)
		"""
		return np.vstack([self.word_average(sent) for sent in docs])

In [4]:
fb1 = mongodb('text_mining','facebook_little_drunk','simi','simi')
fb1.head(3)

,_id,index,time,content,comments
0,603c7f32d723d5c04c4d7755,0,2021-01-27 16:24:00,"[蒂, 芬妮, 月亮, 寶石, 麝香, 氣泡, 葡萄酒, 還配, 高雅, 氣泡, 酒杯, 口...","[{'index': 0, 'created_at': 2021-01-27 21:52:0..."
1,603c7f32d723d5c04c4d7756,1,2021-01-27 15:32:00,"[Highland, Journey, Blended, Malt, 46.2%, 獵人, ...","[{'index': 0, 'created_at': 2021-01-27 15:33:0..."
2,603c7f32d723d5c04c4d7757,2,2021-01-26 21:29:00,"[Taiwind, Beer, 暖果, 荔枝, IPA, 7%, 這瓶, 暖果, 一倒, 非...","[{'index': 0, 'created_at': 2021-01-27 22:00:0..."


In [5]:
doc = []
for i in fb1['content'][:3]:
    for x in i:
        doc.append(i)
# doc = fb1['content']

In [6]:
from gensim.models.word2vec import Word2Vec
word_model = Word2Vec(doc, min_count=2, size=100, window=5, iter=100)

In [7]:
from collections import namedtuple, defaultdict

# from Tfidf_em_vec import TfidfEmbeddingVectorizer
tfidf_vec_tr = TfidfEmbeddingVectorizer(word_model)
tfidf_vec_tr.fit(doc)  # fit tfidf model first
tfidf_doc_vec = tfidf_vec_tr.transform(doc)##文本向量

In [13]:
tfidf_doc_vec

TypeError: 'tuple' object is not callable

In [ ]:
from wordcloud import WordCloud
for i in range(0,5): 
    text = r202003.loc[i].content
 
    wc = WordCloud(font_path="NotoSansCJKtc-Black.otf", #設定字體
         background_color="white", #背景顏色
         max_words = 2000,    #文字雲顯示最大詞數
         stopwords = stopwords)  #刪除停用字

wc.generate(text) 

plt.imshow(wc)
plt.axis("off")
plt.savefig('2020年3月UDN文字趨勢文字雲.png')
plt.figure(figsize=(10,6), dpi = 100)
plt.show()
